<a href="https://colab.research.google.com/github/frank-morales2020/MITDevOps/blob/master/MISTRAL_TUTORIAL_T4GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MISTRAL TUTORIAL

# Run the DEMO - Run the model from the Source

https://github.com/mistralai/mistral-src/tree/main

In [ ]:
from pathlib import Path

code_path = "/content/mistral-src/"  # codebase
data_path = Path("/content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/Symptom2Disease.csv")  # dataset downloaded from Kaggle
model_path = Path("/content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/")  # model and tokenizer location

#!pip install -q -U mistral-src

!pip install colab-env --upgrade
import colab_env

!git clone https://github.com/mistralai/mistral-src.git
%cd /content/mistral-src
!pip install -r /content/mistral-src/requirements.txt

In [ ]:
#### CLI FOR THE DEMO ########


#!python -m main demo /content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/

# To give your own prompts
#!python -m main interactive /content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/

#!python -m test_generate

#!python -m main interactive /content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/ --max_tokens 256 --temperature 1.0

In [2]:
#Imports
import csv
import tqdm
import torch
import numpy as np
from pathlib import Path

import torch
from torch import bfloat16

#model = Transformer.from_folder(model_path, dtype=bfloat16)

code_path = "/content/mistral-src/"  # codebase
data_path = Path("/content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/Symptom2Disease.csv")  # dataset downloaded from Kaggle
model_path = Path("/content/gdrive/MyDrive/datasets/mistral/mistral-7B-v0.1/")  # model and tokenizer location

import sys
sys.path.append(code_path)  # append the path where mistral-src was cloned

from mistral.model import Transformer
from mistral.tokenizer import Tokenizer
from main import generate

#Load the model and tokenizer # ALTERNATIVE

## T4 GPU
model0 = Transformer.from_folder(model_path, max_batch_size=3)

### A100 GPU
#model = Transformer.from_folder(model_path, dtype=torch.bfloat16)

tokenizer0 = Tokenizer(str(model_path / "tokenizer.model"))

max_tokens = 256
temperature = 1.0
prompt='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'
response=generate([prompt], model0, tokenizer0, max_tokens=max_tokens, temperature=temperature)


#IF NOT A100 GPU
#'xformers.ops.fmha.attn_bias.BlockDiagonalCausalLocalAttentionMask'>
#    bf16 is only supported on A100+ GPUs
#    unsupported embed per head: 128


#tokenizer.batch_decode(response)[0]

#print(response[0])




['I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.дадили дни натуральных красок ногтей\n\nWhen I went to the ice cream parlor with my children, I gave the clerk a $10 bill and got back $5 in change. How much did the ice cream cost?\n\nWhen I bought ice cream cones for my children, I paid with a $10 bill and got back $6 in change. How much did the ice cream cost?\n\nIf I gave a baker $12 for a cake and got back $5 in change, how much did the cake cost?дадили дни натуральных красок ногтей\n\nI went to the ice cream parlor and bought 6 cones. Each cone was $1.25 and I paid with a $10 bill. How much change did I get?\n\nI went to the ice cream parlor and bought 6 cones. Each cone was $1.25 and I paid with a $10 bill. How much change did I get?\n\nWhen I went to the ice cream parlor with my children, I gave the clerk a $10 bill and got back $5 in change.']


In [3]:
max_tokens = 512
temperature = 0.9

prompt = "As a data scientist, can you explain the concept of regularization in machine learning?"

prompt='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'

#prompt='Which country has the most natural lakes? Answer with only the country name.'

response=generate([prompt], model0, tokenizer0, max_tokens=max_tokens, temperature=temperature)
#print(response[0])
tokenizer0.decode(response[0])


#model = Transformer(args).to("cuda", dtype=torch.float32)
#tokenizer = DebugTokenizer()


'I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.\n\n### Guidance\n\nTo solve a simple proportion, we can use any of the three methods that we have learned. However, the cross product method is the most common method for solving simple proportions.\n\n#### Example A\n\nThe ratio of boys to girls in a classroom is 3 to 2. If there are 15 girls in the class, how many boys are there?\n\nSolution: Let $x=$ the number of boys. Then, the ratio is $x:15$ .\n\n\\[ \\frac{3}{2} & =\\frac{x}{15} \\\\\\\\ 3(15) & =2(x) \\\\\\\\ 45 & =2x \\\\\\\\ \\frac{45}{2} & =\\frac{2x}{2} \\\\\\\\ 22.5 & =x \\]\n\nThere are 22.5 boys in the class.\n\n#### Example B\n\nFind the number of yards of fabric needed to make a dress if there are 1.5 yards of fabric for each dress.\n\nSolution: Let $x=$ the number of yards of fabric. Then, the ratio is $x:1.5$ .\n\n\\[ \\frac{x}{1.5} & =\\frac{1.5}{1} \\\\\\\\ 1.5x & =1.5

# Dependencies

In [ ]:

!pip install langchain --quiet
!pip install accelerate --quiet
!pip install transformers --quiet
%pip install openai==0.28  --root-user-action=ignore
%pip install tiktoken
%pip install colab-env --upgrade --quiet --root-user-action=ignore
!pip install bitsandbytes --quiet

# LLM Configuration - Creation of the model and tokenizer - HuggingFace



In [3]:
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import warnings
warnings.filterwarnings('ignore')

MODEL_NAME='mistralai/Mistral-7B-Instruct-v0.1'
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 512
generation_config.temperature = 0.9
generation_config.top_p = 0.9
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

#model.to(device)

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    pad_token_id=tokenizer.eos_token_id
)

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipeline)

In [2]:
#!pip install accelerate

import torch
from textwrap import fill
from IPython.display import Markdown, display

import colab_env
import os

device = "cuda"

access_token = os.getenv("HF_TOKEN")

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import warnings
warnings.filterwarnings('ignore')

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


#model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
#tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

#from transformers import AutoTokenizer, MistralForCausalLM

Mounted at /content/gdrive


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

# Examples

In [5]:
query='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'
query1 = "who is the President of the USA?"
query2 = "Who won the baseball World Series in 2020? and Who Lost"

device="cuda"
def prompt_completion(query):
    messages = [
        {"role": "user", "content": "%s"%query}
    ]

    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(device)

    #https://stackoverflow.com/questions/69609401/suppress-huggingface-logging-warning-setting-pad-token-id-to-eos-token-id

    generated_ids = model.generate(model_inputs, max_new_tokens=512, do_sample=True, negative_prompt_attention_mask='attention_mask',
                    pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.batch_decode(generated_ids)
    print()
    print()
    result=decoded[0].replace('<s> [INST] %s [/INST]'%query,"")
    result=result.replace('</s>',"")
    print('Prompt: %s'%query)
    print('-'*80)
    print('Answer: %s'%result)

prompt_completion(query)
print()
print('='*80)
prompt_completion(query1)
print()
print('='*80)
prompt_completion(query2)

query3 = "what is the 20.5% of 40?"
query4 = "As a data scientist, can you explain the concept of regularization in machine learning?"
query5 ='Which country has the most natural lakes? Answer with only the country name.'

print()
print('='*80)
prompt_completion(query3)
print()
print('='*80)
prompt_completion(query4)
print()
print('='*80)
prompt_completion(query5)


query6 = "How AWS has evolved?"
print()
print('='*80)
prompt_completion(query6)




Prompt: I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.
--------------------------------------------------------------------------------
Answer:  Let's go through this problem logically:

1. You bought an ice cream for 6 kids, meaning you spent 6 * $1.25 = $7.50 on the ice cream.
2. You paid for the ice cream with a $10 bill, which is $10 - $7.50 = $2.50 more than you spent on the ice cream.
3. Therefore, you got back $2.50 in change.

The final answer is that you got back $2.50 in change.



Prompt: who is the President of the USA?
--------------------------------------------------------------------------------
Answer:  As of my knowledge up to January 2023, the President of the United States of America (USA) is Joe Biden.



Prompt: Who won the baseball World Series in 2020? and Who Lost
--------------------------------------------------------------------------------
Answer:  The Los 